# Capstone Project
This is the Coursera Data Science Capstone Project. This is the 9. and last of the 9-Parts Data Science Specialization Path.

#### 1. Data Wrangling
#### 2.

Firstly we download the necessary  Libraries

In [131]:
!pip install geopy
!pip install geocoder
!pip install folium

import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.metrics import silhouette_score

%matplotlib notebook

print('All libraries imported.')

All libraries imported.


We build the code to scrape the following Wikipedia page; https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
in order to obtain the data that we will work on.<p>
List of postal codes of Canada: M

In [133]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)

len(df)

3

0. Index is our dataframe and the dataframe is consist of three columns: PostalCode, Borough, and Neighborhood

In [134]:
df1=df[0]
df1.tail()

,Postal Code,Borough,Neighbourhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


In our Dataframe there are more than one neighborhood can exist in one postal code area.
For example, we can easily notice that M5A is listed twice and has two neighborhoods:
Harbourfront and Regent Park.

If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [136]:
df1.loc[df1['Neighbourhood'] == "Not assigned", ['Neighbourhood']] = df1['Borough']

#df1.loc[(df1.Neighbourhood == 'Not assigned'),'Neighbourhood']=df1.Borough
df1

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [137]:
for index, row in df1.iterrows():
    if df1.loc[index, "Neighbourhood"] == "Not assigned" :
        df1.loc[index, "Neighbourhood"] = df1.loc[index, "Borough"]

df1.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [130]:
df1.shape

(180, 3)

We read csv Coordinate values like Latitude and Longitude from url

In [141]:
dfc=pd.read_csv("http://cocl.us/Geospatial_data")

dfc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then we merge two dataframe List of postal codes of Canada as df1 and Coordinates as dfc 

In [143]:
dfm = pd.merge(df1, dfc, on="Postal Code")
dfm

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
